1 - import

2 - data cleaning

3 - create feature flag

# import

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import os 

#Import csv files

In [2]:
path = r'/Users/amritambe/Desktop/CareerF Python/Insta Cart Analysis'
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'))
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'original Data', 'products.csv'))

#PRODUCT.CSV

In [3]:
#drop null values and compare changes

# Data Cleaning

In [4]:
df_prods_ = df_prods[df_prods['product_name'].notna()]
#df_prods.shape
print(df_prods_.shape, df_prods.shape)

(49677, 5) (49693, 5)


In [5]:
#create df with null values

In [6]:
df_nan = df_prods[df_prods['product_name'].isnull()]
df_nan.shape

(16, 5)

In [7]:
#for loop to check for homogenous data types

In [8]:
for col in df_prods.columns.tolist():
    weird = (df_prods[[col]].applymap(type) != df_prods[[col]].iloc[0].apply(type)).any(axis=1)
    if len(df_prods[weird] > 1):
        print(col)


product_name


In [9]:
#identify duplicated rows

In [10]:
df_dups = df_prods_[df_prods_.duplicated()]
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [11]:
#compare changes

In [12]:
df_prods_.shape, df_prods_.drop_duplicates().shape

((49677, 5), (49672, 5))

In [13]:
#create new df cleaned of null and duplicates

In [14]:
df_prods_clean = df_prods_.drop_duplicates()
df_prods_clean.shape

(49672, 5)

#ORDERS.CSV

In [15]:
#check for duplicates and null values in orders.csv

In [16]:
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_between_last_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


The metrics for orders.csv seem to be correct. the order_number column is a little ambiguous, but it pertains to the number of orders for a particular user_id. order_number should be changed to customer_order_number, and user_id should be customer_id. 

In [17]:
df_ords.rename(columns={'order_number' : 'customer_order_number', 'user_id' : 'customer_id' }, inplace=True)

In [18]:
df_ords.head()

,Unnamed: 0,order_id,customer_id,customer_order_number,orders_day_of_week,order_hour_of_day,days_between_last_order
0,0,2539329,1,1,2,8,NaN
1,1,2398795,1,2,3,7,15.0
2,2,473747,1,3,3,12,21.0
3,3,2254736,1,4,4,7,29.0
4,4,431534,1,5,4,15,28.0


In [19]:
#check for homogenous data with a for loop 

In [20]:
for col in df_ords.columns.tolist():
    weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis=1)
    if len(df_ords[weird] > 1):
        print(col)


In [21]:
#no output means the data types are correct

In [22]:
#check for nulls and duplicates

In [23]:
df_ords.isnull().sum()

Unnamed: 0                      0
order_id                        0
customer_id                     0
customer_order_number           0
orders_day_of_week              0
order_hour_of_day               0
days_between_last_order    206209
dtype: int64

In [24]:
df_ords.duplicated().sum()

0

#no duplicates in orders.csv, but the days_between_last_order column has 206,209 null values- about 6% of the total. This is likely due to the customer's first order not having any prior data. I want to create a new boolean column for identifying the first order for a customer. 

In [25]:
#Create first_order column

# create feature flag

In [31]:
df_ords['first_order'] = df_ords['days_between_last_order'].isnull()
df_ords.isnull().sum()

Unnamed: 0                      0
order_id                        0
customer_id                     0
customer_order_number           0
orders_day_of_week              0
order_hour_of_day               0
days_between_last_order    206209
first_order                     0
dtype: int64

In [27]:
#export df_ords and df_prods to csv files into prepared data

In [32]:
df_prods.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'Product_cleaned.csv'))
df_ords.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'Orders_cleaned.csv'))
